In [ ]:
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import configparser
import json

from boto3_utils import (
    get_bucket,
    get_subbucket,
)

In [ ]:
config = configparser.ConfigParser()
with open('dwh.cfg') as inp:
    config.read_file(inp)

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

HOST                   = config.get('CLUSTER', 'HOST')
CLUSTER_NAME           = config.get('CLUSTER', 'CLUSTER_NAME')
DWH_CLUSTER_TYPE       = config.get('CLUSTER', 'DWH_CLUSTER_TYPE')
DWH_NUM_NODES          = config.get('CLUSTER', 'DWH_NUM_NODES')
DWH_NODE_TYPE          = config.get('CLUSTER', 'DWH_NODE_TYPE')

DB_NAME                = config.get('CLUSTER', 'DB_NAME')
DB_USER                = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD            = config.get('CLUSTER', 'DB_PASSWORD')
DB_PORT                = config.get('CLUSTER', 'DB_PORT')

ARN                    = config.get('IAM_ROLE', 'ARN')
IAM_ROLE_NAME          = config.get('IAM_ROLE', 'IAM_ROLE_NAME')

S3_LOG_DATA            = config.get('S3', 'LOG_DATA')
S3_LOG_JSONPATH        = config.get('S3', 'LOG_JSONPATH')
S3_SONG_DATA           = config.get('S3', 'SONG_DATA')

In [ ]:
common_kwargs = {
    'region_name': 'us-west-2',
    'aws_access_key_id': KEY,
    'aws_secret_access_key': SECRET,
}

ec2 = boto3.resource(
    'ec2',
    **common_kwargs,
)

s3_resource = boto3.resource(
    's3',
    **common_kwargs,
)

iam = boto3.client(
    'iam',
    **common_kwargs,
)

redshift = boto3.client(
    'redshift',
    **common_kwargs,
)

## Let's first check out the log data

In [ ]:
bucket = s3_resource.Bucket(get_bucket(S3_LOG_DATA))
for obj in bucket.objects.filter(Prefix=get_subbucket(S3_LOG_DATA)):
    print(obj)

In [ ]:
json_filename = obj.key.split('/')[-1]
s3_resource.meta.client.download_file(
    obj.bucket_name,
    obj.key,
    json_filename,
)

with open(json_filename) as inp:
    for line in inp.readlines():
        log_json = json.loads(line)
        break
    
log_json

## Now, let's check out the song data

In [ ]:
bucket = s3_resource.Bucket(get_bucket(S3_SONG_DATA))
for i, obj in enumerate(bucket.objects.filter(Prefix=get_subbucket(S3_SONG_DATA))):
    print(obj)
    if i == 10:
        break

In [ ]:
json_filename = obj.key.split('/')[-1]
s3_resource.meta.client.download_file(
    obj.bucket_name,
    obj.key,
    json_filename,
)

with open(json_filename) as inp:
    for line in inp.readlines():
        log_json = json.loads(line)
        break
    
log_json

## Finally, let's see what's in the log data json path

In [ ]:
bucket = s3_resource.Bucket(get_bucket(S3_LOG_JSONPATH))
for obj in bucket.objects.filter(Prefix=get_subbucket(S3_LOG_JSONPATH)):
    print(obj)

In [ ]:
S3_LOG_JSONPATH

In [ ]:
json_filename = S3_LOG_JSONPATH.split('/')[-1]
s3_resource.meta.client.download_file(
    get_bucket(S3_LOG_JSONPATH),
    get_subbucket(S3_LOG_JSONPATH),
    json_filename,
)

In [ ]:
with open(json_filename) as inp:
    log_json = json.load(inp)
    
log_json

# Creating an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
if ARN:
    roleArn = ARN
else:    
    iam.get_role(
        RoleName=IAM_ROLE_NAME
    )['Role']['Arn']

    iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = 'Allows Redshift clusters to call AWS services on your behalf.',
        AssumeRolePolicyDocument=json.dumps({
            'Statement': [{
                'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {
                    'Service': 'redshift.amazonaws.com'
                }
            }],
            'Version': '2012-10-17'
        })
    )

    iam.attach_role_policy(
        RoleName=IAM_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
    )['ResponseMetadata']['HTTPStatusCode']
    
    roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

# Creating a Redshift cluster

In [ ]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_NAME,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,

        #Roles (for s3 access)
        IamRoles=[ARN]  
    )
except ClientError as e:
    print(e)

In [ ]:
myClusterProps = redshift.describe_clusters(
    ClusterIdentifier=CLUSTER_NAME
)['Clusters'][0]

# Opening an incoming TCP port to access the cluster endpoint

In [ ]:
vpc = ec2.Vpc(id=myClusterProps['VpcId'])
defaultSg = list(vpc.security_groups.all())[0]
# print(defaultSg)
try:
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except ClientError as e:
    print(e)

### Connect to the cluster

In [ ]:
if not HOST:
    HOST = myClusterProps['Endpoint']['Address']

In [ ]:
%load_ext sql

In [ ]:
conn_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:{DB_PORT}/{DB_NAME}'
# print(conn_string)
%sql $conn_string

In [ ]:
%%sql

CREATE SCHEMA IF NOT EXISTS sparkify;
SET search_path TO sparkify;

This is an example event JSON:
```json
{'artist': 'Stephen Lynch',
 'auth': 'Logged In',
 'firstName': 'Jayden',
 'gender': 'M',
 'itemInSession': 0,
 'lastName': 'Bell',
 'length': 182.85669,
 'level': 'free',
 'location': 'Dallas-Fort Worth-Arlington, TX',
 'method': 'PUT',
 'page': 'NextSong',
 'registration': 1540991795796.0,
 'sessionId': 829,
 'song': "Jim Henson's Dead",
 'status': 200,
 'ts': 1543537327796,
 'userAgent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)',
 'userId': '91'}
```
and let's remember:
```json
{'jsonpaths': ["$['artist']",
  "$['auth']",
  "$['firstName']",
  "$['gender']",
  "$['itemInSession']",
  "$['lastName']",
  "$['length']",
  "$['level']",
  "$['location']",
  "$['method']",
  "$['page']",
  "$['registration']",
  "$['sessionId']",
  "$['song']",
  "$['status']",
  "$['ts']",
  "$['userAgent']",
  "$['userId']"]}
```

In [ ]:
%%sql

DROP TABLE IF EXISTS event_staging;

CREATE TABLE event_staging (
    artist          TEXT,
    auth            TEXT        NOT NULL,
    firstName       TEXT,
    gender          CHAR(1),
    itemInSession   INTEGER     NOT NULL,
    lastName        TEXT,
    length          NUMERIC,
    level           TEXT        NOT NULL,
    location        TEXT,
    method          TEXT        NOT NULL,
    page            TEXT        NOT NULL,
    registration    NUMERIC,
    sessionId       INTEGER     NOT NULL,
    song            VARCHAR,
    status          INTEGER     NOT NULL,
    ts              NUMERIC     NOT NULL,
    userAgent       VARCHAR,
    userId          INTEGER
);

In [ ]:
%%sql

COPY event_staging FROM 's3://udacity-dend/log_data'
IAM_ROLE 'arn:aws:iam::474811394246:role/dwhRole'
FORMAT AS JSON 's3://udacity-dend/log_json_path.json';

After several trials-and-errors, the above worked.

What really helped was taking a peek into the following table:

In [ ]:
%%sql

SELECT starttime, err_code, err_reason
FROM stl_load_errors
ORDER BY starttime DESC
LIMIT 10;

Let's turn the `COPY` statement into a function:

In [ ]:



query = get_copy_json_query('event_staging', S3_LOG_DATA, ARN, S3_LOG_JSONPATH)

%sql $query

OK, now let's remind ourselves how a song JSON looks like:
```json
{'artist_id': 'ARGE7G11187FB37E05',
 'artist_latitude': None,
 'artist_location': 'Brooklyn, NY',
 'artist_longitude': None,
 'artist_name': 'Cyndi Lauper',
 'duration': 240.63955,
 'num_songs': 1,
 'song_id': 'SONRWUU12AF72A4283',
 'title': 'Into The Nightlife',
 'year': 2008}
```

In [ ]:
%%sql

DROP TABLE IF EXISTS song_staging;

CREATE TABLE song_staging (
    artist_id        TEXT      NOT NULL,
    artist_latitude  TEXT,
    artist_longitude TEXT,
    artist_location  TEXT,
    artist_name      TEXT      NOT NULL,
    duration         NUMERIC   NOT NULL,
    num_songs        INTEGER   NOT NULL,
    song_id          TEXT      NOT NULL,
    title            TEXT      NOT NULL,
    year             INTEGER   NOT NULL
);

In [ ]:
query = get_copy_json_query('song_staging', S3_SONG_DATA, ARN)

%sql $query

In [ ]:
%%sql

DROP TABLE IF EXISTS songplays CASCADE;

CREATE TABLE songplays (
  songplay_id    INTEGER         IDENTITY(0, 1)     PRIMARY KEY,
  start_time     TIMESTAMP       NOT NULL,
  user_id        INTEGER         NOT NULL,
  level          TEXT            NOT NULL,
  song_id        TEXT,
  artist_id      TEXT,
  session_id     INTEGER         NOT NULL,
  location       TEXT,
  user_agent     TEXT            NOT NULL
);

In [ ]:
%%sql

INSERT INTO songplays (
    start_time, user_id, level, song_id, artist_id,
    session_id, location, user_agent
)
SELECT
    timestamp 'epoch' + es.ts / 1000 * interval '1 second' AS start_time,
    es.userId                                              AS user_id,
    es.level,
    ss.song_id,
    ss.artist_id,
    es.sessionId as session_id,
    es.location,
    es.userAgent as user_agent
FROM
    event_staging AS es
JOIN
    song_staging AS ss ON (
        es.song = ss.title AND es.artist = ss.artist_name
    )
WHERE
    es.page = 'NextSong';

In [ ]:
%sql SELECT * FROM songplays LIMIT 2;

### It works!
Now, let's populate the rest of the tables.

In [ ]:
%%sql

DROP TABLE IF EXISTS users;

CREATE TABLE users (
  user_id        INTEGER                            PRIMARY KEY,
  first_name     TEXT            NOT NULL,
  last_name      TEXT            NOT NULL,
  gender         CHAR(1)         NOT NULL,
  level          TEXT            NOT NULL
);

INSERT INTO users (
    user_id, first_name, last_name,
    gender, level
)
SELECT
    es.userId,
    es.firstName,
    es.lastName,
    es.gender,
    es.level
FROM
    event_staging AS es
JOIN (
    SELECT
        max(ts) AS ts,
        userId
    FROM
        event_staging
    WHERE
        page = 'NextSong'
    GROUP BY
        userId
    ) AS max_ts_for_user
ON (
    es.userId = max_ts_for_user.userId AND es.ts = max_ts_for_user.ts
);

In [ ]:
%sql SELECT * FROM users LIMIT 2;

In [ ]:
%%sql

DROP TABLE IF EXISTS songs;

CREATE TABLE songs (
  song_id        TEXT                               PRIMARY KEY,
  title          TEXT            NOT NULL,
  artist_id      TEXT            NOT NULL,
  year           INTEGER         NOT NULL,
  duration       NUMERIC         NOT NULL
);

INSERT INTO songs
SELECT
    song_id,
    title,
    artist_id,
    year,
    duration
FROM
    song_staging;

In [ ]:
%sql SELECT * from songs LIMIT 2;

In [ ]:
%%sql

DROP TABLE IF EXISTS time;

CREATE TABLE time (
  start_time    TIMESTAMP                           PRIMARY KEY,
  hour          INTEGER          NOT NULL,
  day           INTEGER          NOT NULL,
  week          INTEGER          NOT NULL,
  month         INTEGER          NOT NULL,
  year          INTEGER          NOT NULL,
  weekday       INTEGER          NOT NULL
);

INSERT INTO time (
    start_time, hour, day, week,
    month, year, weekday
)
SELECT
    timestamp_eval.start_time                       AS start_time,
    EXTRACT(hour FROM timestamp_eval.start_time)    AS hour,
    EXTRACT(day FROM timestamp_eval.start_time)     AS day,
    EXTRACT(week FROM timestamp_eval.start_time)    AS week,
    EXTRACT(month FROM timestamp_eval.start_time)   AS month,
    EXTRACT(year FROM timestamp_eval.start_time)    AS year,
    EXTRACT(weekday FROM timestamp_eval.start_time) AS weekday
FROM (
    SELECT DISTINCT
        TIMESTAMP 'epoch' + event_staging.ts/1000 * INTERVAL '1 second' AS start_time
    FROM 
        event_staging
    WHERE
        page = 'NextSong'
) timestamp_eval;

In [ ]:
%sql SELECT * FROM time LIMIT 2;

In [ ]:
%%sql

DROP TABLE IF EXISTS artists;

CREATE TABLE artists (
  artist_id     TEXT                                PRIMARY KEY,
  name          TEXT             NOT NULL,
  location      TEXT,
  latitude      NUMERIC,
  longitude     NUMERIC
);

INSERT INTO artists (               
    artist_id, name, location,
    latitude, longitude
)
SELECT DISTINCT
    ss.artist_id                AS artist_id,
    ss.artist_name              AS name,
    ss.artist_location          AS location,
    ss.artist_latitude          AS latitude,
    ss.artist_longitude         AS longitude
FROM
    song_staging AS ss;

# Delete the Redshift cluster and IAM role

In [ ]:
# redshift.delete_cluster(
#     ClusterIdentifier=CLUSTER_NAME,
#     SkipFinalClusterSnapshot=True,
# )

# iam.detach_role_policy(
#     RoleName=IAM_ROLE_NAME,
#     PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess',
# )
# iam.delete_role(RoleName=IAM_ROLE_NAME)